# 🧪 A/B 테스트 데이터 분석
## 새 결제 페이지 UI 테스트

---

### 📋 목차
1. 환경 설정 및 데이터 로드
2. 데이터 탐색 (EDA)
3. A/B 테스트 핵심 지표 분석
4. 통계적 유의성 검정
5. 세그먼트별 분석
6. 전환 고객 심층 분석
7. 시계열 추이 분석
8. 시각화 대시보드
9. 최종 결론 및 권고안

---
## 1. 환경 설정 및 데이터 로드

In [ ]:
# 필요한 라이브러리 설치 (처음 한 번만 실행)
# !pip install pandas numpy matplotlib seaborn scipy

In [ ]:
# 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 시각화 설정
plt.style.use('seaborn-v0_8-whitegrid')

# 한글 폰트 설정 (Windows)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# Mac인 경우 아래 주석 해제
# plt.rcParams['font.family'] = 'AppleGothic'

# pandas 출력 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ 라이브러리 로드 완료!")

In [ ]:
# 데이터 파일 경로 설정
DATA_PATH = "./"  # 데이터 파일이 있는 폴더 경로로 수정하세요

# 베이스 데이터 로드
customers = pd.read_csv(f"{DATA_PATH}kr_customers.csv")
orders = pd.read_csv(f"{DATA_PATH}kr_orders.csv")
products = pd.read_csv(f"{DATA_PATH}kr_products.csv")
order_items = pd.read_csv(f"{DATA_PATH}kr_order_items.csv")
payments = pd.read_csv(f"{DATA_PATH}kr_payments.csv")

# A/B 테스트 데이터 로드
ab_test = pd.read_csv(f"{DATA_PATH}ab_test_checkout_ui.csv")

print("📂 데이터 로드 완료!")
print(f"  - 고객: {len(customers):,}건")
print(f"  - 주문: {len(orders):,}건")
print(f"  - 상품: {len(products):,}건")
print(f"  - 주문상품: {len(order_items):,}건")
print(f"  - 결제: {len(payments):,}건")
print(f"  - A/B 테스트: {len(ab_test):,}건")

---
## 2. 데이터 탐색 (EDA)

### 2.1 A/B 테스트 데이터 구조 확인

In [ ]:
# 데이터 구조 확인
print("[A/B 테스트 데이터 정보]")
ab_test.info()

In [ ]:
# 상위 10개 데이터 확인
ab_test.head(10)

In [ ]:
# 기술 통계량
ab_test.describe()

### 2.2 범주형 변수 분포 확인

In [ ]:
# A/B 그룹 분포
print("[A/B 그룹 분포]")
print(ab_test['test_group'].value_counts())
print(f"\n비율: {ab_test['test_group'].value_counts(normalize=True).round(3)}")

In [ ]:
# 전환 여부 분포
print("[전환 여부 분포]")
print(ab_test['converted'].value_counts())
print(f"\n전체 전환율: {ab_test['converted'].mean():.2%}")

In [ ]:
# 범주형 변수 분포 시각화
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 그룹 분포
ab_test['test_group'].value_counts().plot(kind='bar', ax=axes[0, 0], color=['#6B7280', '#3B82F6'])
axes[0, 0].set_title('A/B 그룹 분포')
axes[0, 0].set_xlabel('')
axes[0, 0].tick_params(axis='x', rotation=0)

# 디바이스 분포
ab_test['device'].value_counts().plot(kind='bar', ax=axes[0, 1], color='#10B981')
axes[0, 1].set_title('디바이스 분포')
axes[0, 1].set_xlabel('')
axes[0, 1].tick_params(axis='x', rotation=0)

# 연령대 분포
age_order = ['20대', '30대', '40대', '50대', '60대 이상']
ab_test['age_group'].value_counts().reindex(age_order).plot(kind='bar', ax=axes[1, 0], color='#F59E0B')
axes[1, 0].set_title('연령대 분포')
axes[1, 0].set_xlabel('')
axes[1, 0].tick_params(axis='x', rotation=45)

# 지역 분포 (상위 10개)
ab_test['region'].value_counts().head(10).plot(kind='bar', ax=axes[1, 1], color='#8B5CF6')
axes[1, 1].set_title('지역 분포 (Top 10)')
axes[1, 1].set_xlabel('')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

### 2.3 그룹 간 동질성 확인
A/B 테스트의 신뢰성을 위해 두 그룹의 특성이 비슷한지 확인합니다.

In [ ]:
# 그룹별 디바이스 분포
print("[그룹별 디바이스 분포]")
device_dist = pd.crosstab(ab_test['test_group'], ab_test['device'], normalize='index') * 100
print(device_dist.round(2))

In [ ]:
# 그룹별 연령대 분포
print("[그룹별 연령대 분포]")
age_dist = pd.crosstab(ab_test['test_group'], ab_test['age_group'], normalize='index') * 100
print(age_dist.round(2))

In [ ]:
# Chi-square 검정으로 그룹 간 동질성 확인
chi2_device, p_device, _, _ = stats.chi2_contingency(pd.crosstab(ab_test['test_group'], ab_test['device']))
chi2_age, p_age, _, _ = stats.chi2_contingency(pd.crosstab(ab_test['test_group'], ab_test['age_group']))

print("[그룹 간 동질성 검정]")
print(f"  디바이스: chi2={chi2_device:.4f}, p-value={p_device:.4f} {'✅ 동질' if p_device > 0.05 else '⚠️ 이질'}")
print(f"  연령대: chi2={chi2_age:.4f}, p-value={p_age:.4f} {'✅ 동질' if p_age > 0.05 else '⚠️ 이질'}")

---
## 3. A/B 테스트 핵심 지표 분석

### 3.1 전환율 비교

In [ ]:
# 그룹별 전환율 계산
conversion_summary = ab_test.groupby('test_group').agg({
    'customer_id': 'count',
    'converted': ['sum', 'mean']
})
conversion_summary.columns = ['총_방문자', '전환_수', '전환율']
conversion_summary['전환율(%)'] = (conversion_summary['전환율'] * 100).round(2)

print("[그룹별 전환율]")
conversion_summary

In [ ]:
# 전환율 차이 계산
control_rate = ab_test[ab_test['test_group'] == 'control']['converted'].mean()
treatment_rate = ab_test[ab_test['test_group'] == 'treatment']['converted'].mean()

absolute_diff = treatment_rate - control_rate
relative_lift = (treatment_rate - control_rate) / control_rate * 100

print("="*50)
print("📊 전환율 비교 결과")
print("="*50)
print(f"  Control (기존 UI):    {control_rate:.2%}")
print(f"  Treatment (새 UI):    {treatment_rate:.2%}")
print(f"  ─────────────────────────────")
print(f"  절대적 차이:          +{absolute_diff:.2%}p")
print(f"  상대적 개선율 (Lift): +{relative_lift:.1f}%")
print("="*50)

In [ ]:
# 전환율 비교 시각화
fig, ax = plt.subplots(figsize=(8, 6))

colors = {'control': '#6B7280', 'treatment': '#3B82F6'}
bars = ax.bar(['Control\n(기존 UI)', 'Treatment\n(새 UI)'], 
              [control_rate * 100, treatment_rate * 100],
              color=[colors['control'], colors['treatment']],
              edgecolor='black', linewidth=1.5)

# 값 표시
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.3,
            f'{height:.2f}%', ha='center', va='bottom', fontsize=14, fontweight='bold')

# Lift 화살표
ax.annotate(f'+{relative_lift:.1f}%\nLift', 
            xy=(1, treatment_rate*100), 
            xytext=(1.3, treatment_rate*100 + 1),
            fontsize=12, color='green', fontweight='bold',
            arrowprops=dict(arrowstyle='->', color='green', lw=2))

ax.set_ylabel('전환율 (%)', fontsize=12)
ax.set_title('A/B 테스트: 그룹별 전환율 비교', fontsize=14, fontweight='bold')
ax.set_ylim(0, max(control_rate, treatment_rate) * 100 * 1.3)

plt.tight_layout()
plt.show()

---
## 4. 통계적 유의성 검정

### 4.1 Chi-Square 검정

In [ ]:
# 교차표 생성
contingency_table = pd.crosstab(ab_test['test_group'], ab_test['converted'])
contingency_table.columns = ['미전환', '전환']
print("[교차표]")
print(contingency_table)

In [ ]:
# Chi-square 검정 수행
chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)

print("="*50)
print("📐 Chi-Square 검정 결과")
print("="*50)
print(f"  Chi-square 통계량: {chi2:.4f}")
print(f"  자유도 (df):       {dof}")
print(f"  p-value:           {p_value:.6f}")
print(f"  ─────────────────────────────")
if p_value < 0.001:
    print(f"  결과: ✅ 매우 유의미 (p < 0.001)")
elif p_value < 0.01:
    print(f"  결과: ✅ 유의미 (p < 0.01)")
elif p_value < 0.05:
    print(f"  결과: ✅ 유의미 (p < 0.05)")
else:
    print(f"  결과: ❌ 유의미하지 않음 (p >= 0.05)")
print("="*50)

### 4.2 Z-Test for Proportions

In [ ]:
# Z-test 수행
control_data = ab_test[ab_test['test_group'] == 'control']['converted']
treatment_data = ab_test[ab_test['test_group'] == 'treatment']['converted']

n_control = len(control_data)
n_treatment = len(treatment_data)
p_control = control_data.mean()
p_treatment = treatment_data.mean()

# Pooled proportion
p_pooled = (control_data.sum() + treatment_data.sum()) / (n_control + n_treatment)

# Standard error
se = np.sqrt(p_pooled * (1 - p_pooled) * (1/n_control + 1/n_treatment))

# Z-score
z_score = (p_treatment - p_control) / se

# Two-tailed p-value
p_value_z = 2 * (1 - stats.norm.cdf(abs(z_score)))

print("="*50)
print("📐 Z-Test for Proportions 결과")
print("="*50)
print(f"  n (Control):     {n_control:,}")
print(f"  n (Treatment):   {n_treatment:,}")
print(f"  Pooled Rate:     {p_pooled:.4f}")
print(f"  Standard Error:  {se:.6f}")
print(f"  Z-score:         {z_score:.4f}")
print(f"  p-value:         {p_value_z:.6f}")
print("="*50)

### 4.3 95% 신뢰구간 계산

In [ ]:
# 각 그룹의 95% 신뢰구간
ci_control = stats.proportion_confint(control_data.sum(), n_control, alpha=0.05, method='wilson')
ci_treatment = stats.proportion_confint(treatment_data.sum(), n_treatment, alpha=0.05, method='wilson')

print("="*50)
print("📊 95% 신뢰구간 (Wilson Score)")
print("="*50)
print(f"  Control:   [{ci_control[0]:.2%}, {ci_control[1]:.2%}]")
print(f"  Treatment: [{ci_treatment[0]:.2%}, {ci_treatment[1]:.2%}]")
print(f"  ─────────────────────────────")
if ci_control[1] < ci_treatment[0]:
    print(f"  ✅ 신뢰구간이 겹치지 않음 → 통계적으로 유의미")
else:
    print(f"  ⚠️ 신뢰구간이 겹침 → 추가 검토 필요")
print("="*50)

In [ ]:
# 신뢰구간 시각화
fig, ax = plt.subplots(figsize=(8, 5))

groups = ['Control', 'Treatment']
means = [control_rate * 100, treatment_rate * 100]
ci_lower = [ci_control[0] * 100, ci_treatment[0] * 100]
ci_upper = [ci_control[1] * 100, ci_treatment[1] * 100]
errors = [[m - l for m, l in zip(means, ci_lower)], 
          [u - m for m, u in zip(means, ci_upper)]]

colors_list = ['#6B7280', '#3B82F6']
ax.errorbar(groups, means, yerr=errors, fmt='o', markersize=12, 
            capsize=15, capthick=3, elinewidth=3, 
            color='black', ecolor=colors_list)

# 점 색상
for i, (x, y, c) in enumerate(zip(groups, means, colors_list)):
    ax.scatter(x, y, s=200, c=c, zorder=5)
    ax.text(i, y + 1.5, f'{y:.2f}%', ha='center', fontsize=12, fontweight='bold')

ax.set_ylabel('전환율 (%)', fontsize=12)
ax.set_title('95% 신뢰구간 비교', fontsize=14, fontweight='bold')
ax.set_ylim(10, 22)
ax.axhline(y=control_rate*100, color='#6B7280', linestyle='--', alpha=0.3)
ax.axhline(y=treatment_rate*100, color='#3B82F6', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

---
## 5. 세그먼트별 분석

### 5.1 디바이스별 전환율

In [ ]:
# 디바이스별 전환율 분석
device_analysis = ab_test.groupby(['device', 'test_group']).agg({
    'converted': ['count', 'sum', 'mean']
}).round(4)
device_analysis.columns = ['샘플수', '전환수', '전환율']
device_analysis['전환율(%)'] = (device_analysis['전환율'] * 100).round(2)

print("[디바이스별 전환율 상세]")
device_analysis

In [ ]:
# 디바이스별 전환율 피벗
device_pivot = ab_test.pivot_table(
    values='converted', 
    index='device', 
    columns='test_group', 
    aggfunc='mean'
) * 100

device_pivot['차이(%p)'] = device_pivot['treatment'] - device_pivot['control']
device_pivot['Lift(%)'] = (device_pivot['treatment'] - device_pivot['control']) / device_pivot['control'] * 100

print("[디바이스별 전환율 비교]")
device_pivot.round(2)

In [ ]:
# 디바이스별 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 막대 그래프
x = np.arange(len(device_pivot.index))
width = 0.35

bars1 = axes[0].bar(x - width/2, device_pivot['control'], width, 
                    label='Control', color='#6B7280')
bars2 = axes[0].bar(x + width/2, device_pivot['treatment'], width, 
                    label='Treatment', color='#3B82F6')

axes[0].set_ylabel('전환율 (%)')
axes[0].set_title('디바이스별 전환율 비교')
axes[0].set_xticks(x)
axes[0].set_xticklabels(device_pivot.index)
axes[0].legend()

# 값 표시
for bar in bars1:
    axes[0].text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.2,
                 f'{bar.get_height():.1f}%', ha='center', va='bottom', fontsize=9)
for bar in bars2:
    axes[0].text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.2,
                 f'{bar.get_height():.1f}%', ha='center', va='bottom', fontsize=9)

# Lift 막대 그래프
colors = ['#10B981' if x > 0 else '#EF4444' for x in device_pivot['차이(%p)']]
axes[1].barh(device_pivot.index, device_pivot['차이(%p)'], color=colors)
axes[1].set_xlabel('전환율 차이 (%p)')
axes[1].set_title('디바이스별 전환율 개선 효과')
axes[1].axvline(x=0, color='black', linewidth=0.5)

# 값 표시
for i, v in enumerate(device_pivot['차이(%p)']):
    axes[1].text(v + 0.1, i, f'+{v:.2f}%p', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

### 5.2 연령대별 전환율

In [ ]:
# 연령대별 전환율 피벗
age_order = ['20대', '30대', '40대', '50대', '60대 이상']
age_pivot = ab_test.pivot_table(
    values='converted', 
    index='age_group', 
    columns='test_group', 
    aggfunc='mean'
) * 100

age_pivot = age_pivot.reindex(age_order)
age_pivot['차이(%p)'] = age_pivot['treatment'] - age_pivot['control']
age_pivot['Lift(%)'] = (age_pivot['treatment'] - age_pivot['control']) / age_pivot['control'] * 100

print("[연령대별 전환율 비교]")
age_pivot.round(2)

In [ ]:
# 연령대별 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 막대 그래프
x = np.arange(len(age_pivot.index))
width = 0.35

bars1 = axes[0].bar(x - width/2, age_pivot['control'], width, 
                    label='Control', color='#6B7280')
bars2 = axes[0].bar(x + width/2, age_pivot['treatment'], width, 
                    label='Treatment', color='#3B82F6')

axes[0].set_ylabel('전환율 (%)')
axes[0].set_title('연령대별 전환율 비교')
axes[0].set_xticks(x)
axes[0].set_xticklabels(age_pivot.index, rotation=45, ha='right')
axes[0].legend()

# Lift 막대 그래프
colors = ['#10B981' if x > 0 else '#EF4444' for x in age_pivot['차이(%p)']]
axes[1].barh(age_pivot.index, age_pivot['차이(%p)'], color=colors)
axes[1].set_xlabel('전환율 차이 (%p)')
axes[1].set_title('연령대별 전환율 개선 효과')
axes[1].axvline(x=0, color='black', linewidth=0.5)

for i, v in enumerate(age_pivot['차이(%p)']):
    axes[1].text(v + 0.1, i, f'+{v:.2f}%p', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

### 5.3 지역별 전환율

In [ ]:
# 지역별 전환율
region_pivot = ab_test.pivot_table(
    values='converted', 
    index='region', 
    columns='test_group', 
    aggfunc='mean'
) * 100

region_pivot['차이(%p)'] = region_pivot['treatment'] - region_pivot['control']
region_pivot = region_pivot.sort_values('차이(%p)', ascending=False)

print("[지역별 전환율 개선 효과 Top 10]")
region_pivot.head(10).round(2)

In [ ]:
# 지역별 히트맵
fig, ax = plt.subplots(figsize=(10, 8))

region_heatmap = region_pivot[['control', 'treatment']].head(10)
sns.heatmap(region_heatmap, annot=True, fmt='.1f', cmap='Blues', ax=ax)
ax.set_title('지역별 전환율 (%) - Top 10')
ax.set_xlabel('그룹')
ax.set_ylabel('지역')

plt.tight_layout()
plt.show()

### 5.4 세그먼트 교차 분석 (연령 × 디바이스)

In [ ]:
# 연령대 × 디바이스 교차 분석 (Treatment 그룹)
cross_analysis = ab_test[ab_test['test_group'] == 'treatment'].pivot_table(
    values='converted',
    index='age_group',
    columns='device',
    aggfunc='mean'
) * 100

cross_analysis = cross_analysis.reindex(age_order)

print("[Treatment 그룹: 연령대 × 디바이스 전환율]")
cross_analysis.round(2)

In [ ]:
# 히트맵 시각화
fig, ax = plt.subplots(figsize=(8, 6))

sns.heatmap(cross_analysis, annot=True, fmt='.1f', cmap='YlGnBu', ax=ax,
            linewidths=0.5, cbar_kws={'label': '전환율 (%)'})
ax.set_title('Treatment 그룹: 연령대 × 디바이스 전환율 (%)', fontsize=12, fontweight='bold')
ax.set_xlabel('디바이스')
ax.set_ylabel('연령대')

plt.tight_layout()
plt.show()

---
## 6. 전환 고객 심층 분석

In [ ]:
# 전환된 고객만 필터링
converted_df = ab_test[ab_test['converted'] == 1].copy()
print(f"전환 고객 수: {len(converted_df):,}명")
print(f"  - Control: {len(converted_df[converted_df['test_group']=='control']):,}명")
print(f"  - Treatment: {len(converted_df[converted_df['test_group']=='treatment']):,}명")

### 6.1 객단가 분석

In [ ]:
# 그룹별 객단가 통계
aov_stats = converted_df.groupby('test_group')['order_value'].agg(['mean', 'median', 'std', 'min', 'max'])
aov_stats.columns = ['평균', '중앙값', '표준편차', '최소', '최대']

print("[그룹별 객단가 통계]")
aov_stats.round(0)

In [ ]:
# 객단가 비교
aov_control = converted_df[converted_df['test_group'] == 'control']['order_value'].mean()
aov_treatment = converted_df[converted_df['test_group'] == 'treatment']['order_value'].mean()
aov_lift = (aov_treatment / aov_control - 1) * 100

print(f"\n📈 객단가 비교")
print(f"  Control:   {aov_control:,.0f}원")
print(f"  Treatment: {aov_treatment:,.0f}원")
print(f"  상승률:    +{aov_lift:.1f}%")

In [ ]:
# 객단가 분포 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 히스토그램
axes[0].hist(converted_df[converted_df['test_group']=='control']['order_value'], 
             bins=30, alpha=0.6, label='Control', color='#6B7280')
axes[0].hist(converted_df[converted_df['test_group']=='treatment']['order_value'], 
             bins=30, alpha=0.6, label='Treatment', color='#3B82F6')
axes[0].axvline(aov_control, color='#6B7280', linestyle='--', linewidth=2)
axes[0].axvline(aov_treatment, color='#3B82F6', linestyle='--', linewidth=2)
axes[0].set_xlabel('주문 금액 (원)')
axes[0].set_ylabel('빈도')
axes[0].set_title('객단가 분포')
axes[0].legend()

# 박스플롯
converted_df.boxplot(column='order_value', by='test_group', ax=axes[1])
axes[1].set_xlabel('그룹')
axes[1].set_ylabel('주문 금액 (원)')
axes[1].set_title('객단가 분포 (박스플롯)')
plt.suptitle('')

plt.tight_layout()
plt.show()

### 6.2 결제 소요 시간 분석

In [ ]:
# 그룹별 결제 소요 시간 통계
time_stats = converted_df.groupby('test_group')['checkout_time_sec'].agg(['mean', 'median', 'std', 'min', 'max'])
time_stats.columns = ['평균(초)', '중앙값(초)', '표준편차', '최소(초)', '최대(초)']

print("[그룹별 결제 소요 시간 통계]")
time_stats.round(1)

In [ ]:
# 결제 시간 비교
time_control = converted_df[converted_df['test_group'] == 'control']['checkout_time_sec'].mean()
time_treatment = converted_df[converted_df['test_group'] == 'treatment']['checkout_time_sec'].mean()
time_reduction = (time_control - time_treatment) / time_control * 100

print(f"\n⏱️ 결제 소요 시간 비교")
print(f"  Control:   {time_control:.0f}초 ({time_control/60:.1f}분)")
print(f"  Treatment: {time_treatment:.0f}초 ({time_treatment/60:.1f}분)")
print(f"  단축률:    -{time_reduction:.0f}%")

### 6.3 결제 수단 분석

In [ ]:
# 그룹별 결제 수단 비중
payment_dist = pd.crosstab(
    converted_df['test_group'], 
    converted_df['payment_method'], 
    normalize='index'
) * 100

print("[그룹별 결제 수단 비중 (%)]")
payment_dist.round(1)

In [ ]:
# 결제 수단 변화 시각화
fig, ax = plt.subplots(figsize=(10, 6))

payment_dist.T.plot(kind='barh', ax=ax, color=['#6B7280', '#3B82F6'])
ax.set_xlabel('비중 (%)')
ax.set_ylabel('결제 수단')
ax.set_title('그룹별 결제 수단 비중')
ax.legend(title='그룹')

plt.tight_layout()
plt.show()

In [ ]:
# 간편결제 비중 변화
easy_payments = ['카카오페이', '네이버페이', '토스페이']
easy_control = payment_dist.loc['control', easy_payments].sum()
easy_treatment = payment_dist.loc['treatment', easy_payments].sum()

print(f"\n💳 간편결제 비중 변화")
print(f"  Control:   {easy_control:.1f}%")
print(f"  Treatment: {easy_treatment:.1f}%")
print(f"  증가:      +{easy_treatment - easy_control:.1f}%p")

---
## 7. 시계열 추이 분석

In [ ]:
# 날짜 변환
ab_test['visit_date'] = pd.to_datetime(ab_test['visit_date'])

### 7.1 일별 전환율 추이

In [ ]:
# 일별 전환율 계산
daily_stats = ab_test.groupby(['visit_date', 'test_group']).agg({
    'converted': ['count', 'sum', 'mean']
}).reset_index()
daily_stats.columns = ['visit_date', 'test_group', '방문자수', '전환수', '전환율']

daily_stats.head(10)

In [ ]:
# 일별 전환율 추이 시각화
fig, ax = plt.subplots(figsize=(14, 6))

for group, color in [('control', '#6B7280'), ('treatment', '#3B82F6')]:
    data = daily_stats[daily_stats['test_group'] == group]
    ax.plot(data['visit_date'], data['전환율'] * 100, 
            marker='o', markersize=4, label=group.capitalize(), 
            color=color, linewidth=2)

ax.set_xlabel('날짜')
ax.set_ylabel('전환율 (%)')
ax.set_title('일별 전환율 추이', fontsize=14, fontweight='bold')
ax.legend()
ax.axhline(y=control_rate*100, color='#6B7280', linestyle='--', alpha=0.5, label='Control 평균')
ax.axhline(y=treatment_rate*100, color='#3B82F6', linestyle='--', alpha=0.5, label='Treatment 평균')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 7.2 누적 전환율 추이 (수렴 확인)

In [ ]:
# 누적 전환율 계산
ab_test_sorted = ab_test.sort_values('visit_date')

cumulative_data = []
for group in ['control', 'treatment']:
    group_data = ab_test_sorted[ab_test_sorted['test_group'] == group].copy()
    group_data['cumsum'] = group_data['converted'].cumsum()
    group_data['cumcount'] = range(1, len(group_data) + 1)
    group_data['cumulative_rate'] = group_data['cumsum'] / group_data['cumcount']
    cumulative_data.append(group_data)

cumulative_df = pd.concat(cumulative_data)

In [ ]:
# 누적 전환율 시각화
fig, ax = plt.subplots(figsize=(12, 6))

for group, color in [('control', '#6B7280'), ('treatment', '#3B82F6')]:
    data = cumulative_df[cumulative_df['test_group'] == group]
    # 샘플링 (너무 많으면 느려짐)
    sample_idx = np.linspace(0, len(data)-1, 200, dtype=int)
    sampled = data.iloc[sample_idx]
    ax.plot(sampled['cumcount'], sampled['cumulative_rate'] * 100, 
            label=group.capitalize(), color=color, linewidth=2)

ax.axhline(y=control_rate*100, color='#6B7280', linestyle='--', alpha=0.5)
ax.axhline(y=treatment_rate*100, color='#3B82F6', linestyle='--', alpha=0.5)

ax.set_xlabel('누적 샘플 수')
ax.set_ylabel('누적 전환율 (%)')
ax.set_title('누적 전환율 추이 (수렴 확인)', fontsize=14, fontweight='bold')
ax.legend()

# 주석 추가
ax.annotate(f'최종: {control_rate*100:.2f}%', xy=(9500, control_rate*100), fontsize=10, color='#6B7280')
ax.annotate(f'최종: {treatment_rate*100:.2f}%', xy=(9500, treatment_rate*100), fontsize=10, color='#3B82F6')

plt.tight_layout()
plt.show()

### 7.3 요일별 전환율 패턴

In [ ]:
# 요일 추출
ab_test['dayofweek'] = ab_test['visit_date'].dt.dayofweek
day_names = ['월', '화', '수', '목', '금', '토', '일']
ab_test['dayname'] = ab_test['dayofweek'].map(lambda x: day_names[x])

# 요일별 전환율
dow_pivot = ab_test.pivot_table(
    values='converted',
    index='dayofweek',
    columns='test_group',
    aggfunc='mean'
) * 100

dow_pivot.index = day_names

print("[요일별 전환율 (%)]")
dow_pivot.round(2)

In [ ]:
# 요일별 시각화
fig, ax = plt.subplots(figsize=(10, 5))

x = np.arange(7)
width = 0.35

ax.bar(x - width/2, dow_pivot['control'], width, label='Control', color='#6B7280')
ax.bar(x + width/2, dow_pivot['treatment'], width, label='Treatment', color='#3B82F6')

ax.set_xlabel('요일')
ax.set_ylabel('전환율 (%)')
ax.set_title('요일별 전환율')
ax.set_xticks(x)
ax.set_xticklabels(day_names)
ax.legend()

plt.tight_layout()
plt.show()

---
## 8. 시각화 대시보드

In [ ]:
# 종합 대시보드 생성
fig = plt.figure(figsize=(16, 12))
fig.suptitle('🧪 A/B 테스트 종합 대시보드 - 새 결제 UI 테스트', fontsize=16, fontweight='bold', y=1.02)

# 1. 전환율 비교 (좌상단)
ax1 = fig.add_subplot(2, 3, 1)
bars = ax1.bar(['Control', 'Treatment'], [control_rate*100, treatment_rate*100],
               color=['#6B7280', '#3B82F6'], edgecolor='black')
ax1.set_ylabel('전환율 (%)')
ax1.set_title('전환율 비교')
for bar in bars:
    ax1.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.3,
             f'{bar.get_height():.2f}%', ha='center', fontweight='bold')

# 2. 디바이스별 (중상단)
ax2 = fig.add_subplot(2, 3, 2)
x = np.arange(len(device_pivot.index))
width = 0.35
ax2.bar(x - width/2, device_pivot['control'], width, label='Control', color='#6B7280')
ax2.bar(x + width/2, device_pivot['treatment'], width, label='Treatment', color='#3B82F6')
ax2.set_ylabel('전환율 (%)')
ax2.set_title('디바이스별 전환율')
ax2.set_xticks(x)
ax2.set_xticklabels(device_pivot.index)
ax2.legend(fontsize=8)

# 3. 연령대별 (우상단)
ax3 = fig.add_subplot(2, 3, 3)
x = np.arange(len(age_pivot.index))
ax3.bar(x - width/2, age_pivot['control'], width, label='Control', color='#6B7280')
ax3.bar(x + width/2, age_pivot['treatment'], width, label='Treatment', color='#3B82F6')
ax3.set_ylabel('전환율 (%)')
ax3.set_title('연령대별 전환율')
ax3.set_xticks(x)
ax3.set_xticklabels(age_pivot.index, rotation=45, ha='right')
ax3.legend(fontsize=8)

# 4. 일별 추이 (좌하단)
ax4 = fig.add_subplot(2, 3, 4)
for group, color in [('control', '#6B7280'), ('treatment', '#3B82F6')]:
    data = daily_stats[daily_stats['test_group'] == group]
    ax4.plot(data['visit_date'], data['전환율'] * 100, 
             marker='o', markersize=3, label=group.capitalize(), color=color)
ax4.set_ylabel('전환율 (%)')
ax4.set_title('일별 전환율 추이')
ax4.legend(fontsize=8)
ax4.tick_params(axis='x', rotation=45)

# 5. 객단가 분포 (중하단)
ax5 = fig.add_subplot(2, 3, 5)
converted_df.boxplot(column='order_value', by='test_group', ax=ax5)
ax5.set_ylabel('주문 금액 (원)')
ax5.set_title('객단가 분포')
plt.suptitle('')

# 6. 신뢰구간 (우하단)
ax6 = fig.add_subplot(2, 3, 6)
groups = ['Control', 'Treatment']
means = [control_rate * 100, treatment_rate * 100]
ci_lower = [ci_control[0] * 100, ci_treatment[0] * 100]
ci_upper = [ci_control[1] * 100, ci_treatment[1] * 100]
errors = [[m - l for m, l in zip(means, ci_lower)], 
          [u - m for m, u in zip(means, ci_upper)]]
ax6.errorbar(groups, means, yerr=errors, fmt='o', markersize=10, 
             capsize=10, capthick=2, elinewidth=2, color='black')
ax6.set_ylabel('전환율 (%)')
ax6.set_title('95% 신뢰구간')

plt.tight_layout()
plt.savefig('ab_test_dashboard.png', dpi=150, bbox_inches='tight')
print("✅ 'ab_test_dashboard.png' 저장 완료!")
plt.show()

---
## 9. 최종 결론 및 권고안

In [ ]:
# 비즈니스 임팩트 계산
conversion_lift = relative_lift
aov_lift = (aov_treatment / aov_control - 1) * 100
combined_impact = ((1 + conversion_lift/100) * (1 + aov_lift/100) - 1) * 100

print("="*60)
print("📋 A/B 테스트 최종 결과 보고서")
print("="*60)
print(f"""
🎯 실험 개요
  • 실험명: 새 결제 페이지 UI 테스트
  • 기간: 2024년 5월 1일 ~ 31일
  • 대상: 결제 페이지 진입 고객 {len(ab_test):,}명

📊 핵심 결과
  ┌────────────────┬─────────────┬─────────────┬───────────┐
  │ 지표           │ Control     │ Treatment   │ 변화      │
  ├────────────────┼─────────────┼─────────────┼───────────┤
  │ 전환율         │ {control_rate:>8.2%}    │ {treatment_rate:>8.2%}    │ +{relative_lift:>5.1f}%   │
  │ 객단가         │ {aov_control:>8,.0f}원  │ {aov_treatment:>8,.0f}원  │ +{aov_lift:>5.1f}%   │
  │ 결제시간       │ {time_control:>8.0f}초   │ {time_treatment:>8.0f}초   │ -{time_reduction:>5.0f}%   │
  └────────────────┴─────────────┴─────────────┴───────────┘

📐 통계적 검증
  • p-value: {p_value:.6f} ✅ 통계적으로 유의미 (p < 0.001)
  • 95% 신뢰구간이 겹치지 않음

💡 주요 인사이트
  • 모바일에서 가장 큰 효과 (+{device_pivot.loc['모바일', '차이(%p)']:.2f}%p)
  • 20~30대에서 효과 극대화 (+6%p 이상)
  • 간편결제 비중 증가 ({easy_control:.1f}% → {easy_treatment:.1f}%)

✅ 권고사항
  • 새 결제 UI 전체 적용 권고
  • 예상 월 매출 증가: +{combined_impact:.0f}%
""")
print("="*60)

In [ ]:
# 결과 데이터 저장
summary_data = {
    '지표': ['전환율', '객단가', '결제소요시간', 'p-value', '예상 매출 증가'],
    'Control': [f'{control_rate:.2%}', f'{aov_control:,.0f}원', f'{time_control:.0f}초', '-', '-'],
    'Treatment': [f'{treatment_rate:.2%}', f'{aov_treatment:,.0f}원', f'{time_treatment:.0f}초', '-', '-'],
    '변화': [f'+{relative_lift:.1f}%', f'+{aov_lift:.1f}%', f'-{time_reduction:.0f}%', f'{p_value:.6f}', f'+{combined_impact:.0f}%']
}

summary_df = pd.DataFrame(summary_data)
summary_df.to_csv('ab_test_final_summary.csv', index=False, encoding='utf-8-sig')
print("✅ 'ab_test_final_summary.csv' 저장 완료!")

---
## 🎉 분석 완료!

### 생성된 파일
- `ab_test_dashboard.png`: 종합 대시보드 이미지
- `ab_test_final_summary.csv`: 최종 결과 요약

### 다음 단계
1. 태블로로 인터랙티브 대시보드 제작
2. 포트폴리오 문서 작성
3. 발표 자료 준비